# Imports

In [1]:
import mesa
from mesa import Agent, Model
from mesa.time import BaseScheduler

# Agents

In [ ]:
"""
J'imagine percepts comme cela:
percepts = {(x1, y1) : {"agents": [list of agents at pos1],
                        "waste": [list of waste at pos1]
                        "zone": "color of the zone at pos1"}, 
            (x2, y2) : {"agents": [list of agents at pos2],
                        "waste": [list of waste at pos2]
                        "zone": "color of the zone at pos2"},
            ...}
C'est avec cette structu que j'ai commencé à coder deliberate
"""

class GreenRobot(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.knowledge = {"color":"green", "pos": pos,"waste1":None, "waste2": None, "percepts": {}}
        
    def step(self):
        update(self.knowledge, percepts) 
        action = deliberate(self.knowledge) 
        percepts = self.model.do(self,action)

In [ ]:
# not allowed to access any variable outside its argument.
# donne en sortie l'acion à prendre
def deliberate(knowledge):
    if knowledge.color == "green":
        if knowledge["waste1"].color == "yellow": # Carrying a transformed waste. 
            is_right_cell = lambda x: x[0] == knowledge["pos"][0] + 1
            right_cell = [key for key in knowledge["percepts"].keys() if is_right_cell(key)][0]
            if knowledge["percepts"][right_cell]["zone"] == "green": # Is the green robot at the border
                return "right" # If no, transport the transformed waste futher east
            else:
                return "drop" # If yes, drop it here
        else:
            if (knowledge["waste1"] is not None) and (knowledge["waste2"] is not None): # Carrying two green wastes
                return "transform"
            else: # Must find a waste
                if knowledge["percepts"][knowledge["pos"]]["waste"] != []:
                    if knowledge["percepts"][knowledge["pos"]]["waste"][0].color = "green": # Wether the robot stands on a cell with a green waste
                        return "grab" # Grab the waste
                else: # Should move to find a waste
                    possible_positions = []
                    for cell in knowledge["percepts"]:
                        if knowledge["percepts"][cell]["zone"] == "green":
                            # TODO : add more conditions to create a smart decision
                            return possible_positions[random.randint(0, len(possible_positions)-1)] # Next cell to go
    elif knowledge.color == "yellow":
        # TODO

In [ ]:
# mets à jour knowledge à partir de percepts (qui contient les informations de l'environnement)
def update(knowledge, percepts):
    knowledge["percepts"] = percepts

# Objects

In [ ]:
class Radioactivity(Agent):
    def __init__(self, unique_id, model, zone, level):
        super().__init__(unique_id, model)
        self.zone = zone
        self.level = level

In [ ]:
class WasteDisposalZone(Agent):
     def __init__(self, unique_id, model):
            super().__init__(unique_id, model)
            

In [ ]:
class Waste(Agent):
    def __init__(self, unique_id, model, color, pos):
        super().__init__(unique_id, model)
        self.color = color
        self.pos = pos

# Scheduler

# Model

In [ ]:
class RobotMissionModel(Model):
    """Robot mission."""
    def __init__(self, width, height, N_green, N_yellow, N_red, W):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)

        # Instanciate robots
        for i in range(N_green + N_yellow + N_red):
            pos = random.randint(0, width-1), random.randint(0, height-1)
            if i < N_green:
                a = GreenRobot(i, self, pos) 
            elif (i >= N_green) & (i < N_yellow):
                a = YellowRobot(i, self, pos) 
            else:
                a = RedRobot(i, self, pos) 
            self.grid.place_agent(a, pos)
            self.schedule.add(a)
        
        # Instantiate disposal zone
        y_disposal_zone = random.randint(0, height-1)
        agent_waste_disposal_zone = WasteDisposalZone(0, self)
        self.grid.place_agent(agent_waste_disposal_zone, (width-1, y_disposal_zone))
        
        # Instantiate radioactivity
        id_radioactivity_agent = 0
        for x in range(width):
            for y in range(height):
                if (x == width-1) & (y == y_disposal_zone): # Waste Disposal Zone
                    pass
                 elif x <= width//3:
                    r = Radioactivity(id_radioactivity_agent, self, "green", random.uniform(0, 0.33))
                elif (x > witdh//3) & (x <= width*2//3):
                    r = Radioactivity(id_radioactivity_agent, self, "yellow", random.uniform(0.33, 0.66))
                else:
                    r = Radioactivity(id_radioactivity_agent, self, "red", random.uniform(0.66, 1))
                id_radioactivity_agent = id_radioactivity_agent + 1
                self.grid.place_agent(r, pos)
        
        # Instantiate wastes
        id_waste = 0
        for i in range(W_green): 
            pos = random.randint(0, width//3), random.randint(0, height-1)
            w = Waste(id_waste, self, "green", pos)
            self.grid.place_agent(w, pos)
            self.schedule.add(w)
            id_waste = id_waste + 1
                    
    def step(self):
        self.schedule.step()

# Run